In [ ]:
import gzip
import pandas as pd
import numpy as np
import json

In [ ]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

def getDF(path,columns=None,users=None):
    i = 0
    df = {}
    for d in parse(path):
        if columns != None:
            d = {i:d[i] for i in columns}
        if users!=None:
            if d['reviewerID'] in users:
                df[i] = d
                i += 1
        else:
            df[i] = d
            i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
df = getDF('./data/Clothing_Shoes_and_Jewelry.json.gz',{'reviewerID','asin'})

In [ ]:
df.head()

,reviewerID,asin
0,A2IC3NZN488KWK,0871167042
1,A3OT9BYASFGU2X,0871167042
2,A28GK1G2KDXHRP,0871167042
3,A3NFXFEKW8OK0E,0871167042
4,A3I6G5TKBVJEK9,0871167042


In [ ]:
df_filtered = df.drop_duplicates()

In [ ]:
df_filtered.shape, df.shape

((31663536, 2), (32292099, 2))

In [ ]:
users_consumption= df_filtered.groupby('reviewerID').count()

In [ ]:
top_users = users_consumption.loc[users_consumption['asin'] >=5]

In [ ]:
top_users=top_users.sample(100000)

In [ ]:
top_users_set = set(top_users.index.values)

In [ ]:
final_df = getDF('./data/Clothing_Shoes_and_Jewelry.json.gz',users=top_users_set)

In [ ]:
final_df.groupby('reviewerID')['asin'].count().value_counts()[:20]

5     23062
6     17261
7     12361
8      9124
9      6902
10     5201
11     4169
12     3199
13     2695
14     2109
15     1751
16     1456
17     1292
18     1068
19      922
20      755
21      647
22      579
23      530
24      452
Name: asin, dtype: int64

In [ ]:
#final_df.to_parquet('./data/amazon_clothing.parquet')

In [ ]:
final_df.to_csv('./data/amazon_clothing.csv')

In [ ]:
final_df.to_parquet('./data/amazon_clothing.parquet')

In [ ]:
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [ ]:
FindLayerNodesLinear(4,100,5)